# Ex situ GIWAXS processing

# GIWAXS raw data processing & exporting notebook
In this notebook you output xr.DataSets stored as .zarr stores containing all your raw,
remeshed (reciprocal space), and caked CMS GIWAXS data. Saving as a zarr automatically converts the array to a dask array

## Imports

In [ ]:
### Imports:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import xarray as xr
import PyHyperScattering as phs
import fabio
import gc
from tqdm.auto import tqdm  # progress bar loader!

print(f"Don't worry about the above warnings/errors... using PyHyperScattering version: {phs.__version__}!")

## Defining some objects

### Define & check paths

In [ ]:
# I like pathlib for its readability & checkability, it's also necessary for the loadSeries function later on
# Replace the paths with the ones relevant to your data, you can use the ".exists()" method to make sure you defined a path correctly
suitePath = pathlib.Path('/Users/andrew/Library/CloudStorage/OneDrive-UCB-O365/research/data_analysis/giwaxs_suite')
xenocsPath = suitePath.joinpath('raw_data/xenocs')
rawPath = xenocsPath.joinpath('AX_films_01_2024-05-09')
outPath = suitePath.joinpath('processed_data/xenocs')

# Select poni & mask filepaths
# poniFile = xenocsPath.joinpath('xenocs_100sdd.poni')
poniFile = xenocsPath.joinpath('xenocs_120sdd.poni')

# Colormap
cmap = plt.cm.turbo.copy()
cmap.set_bad('black')

In [ ]:
pilatus1m = 0.000172  # cms detector 
eiger2_1m = 0.000075  # xenocs xeuss 3.0 detector

def poni_centers(poniFile, pix_size=eiger2_1m):
    """
    Returns poni center value and the corresponding pixel position. Default pixel size is 172 microns (Pilatus 1M)
    
    Inputs: poniFile as pathlib path object to the poni file
    Outputs: ((poni1, y_center), (poni2, x_center))
    """
    
    with poniFile.open('r') as f:
        lines = list(f.readlines())
    poni1_str = lines[6]
    poni2_str = lines[7]

    poni1 = float(poni1_str.split(' ')[1])
    poni2 = float(poni2_str.split(' ')[1])

    y_center = poni1 / pix_size
    x_center = poni2 / pix_size
        
    return ((poni1, y_center), (poni2, x_center))

display(poni_centers(poniFile))

In [ ]:
[f.name for f in rawPath.glob('*')]

### Define metadata naming scheme & initialize loaders

In [ ]:
sample_folders = sorted(rawPath.glob('A*'))

display(sample_folders)
display([f.name for f in sorted(sample_folders[0].glob('*edf'))])

In [ ]:
# set ex situ metadata filename naming schemes:
# md_naming_scheme = ['material', 'solvent', 'misc', 'stitched', 'scan_ids']
md_naming_scheme = ['film', 'film_number', 'misc', 'scan_id']


# Initalize CMSGIWAXSLoader objects with the above naming schemes
loader = phs.load.CMSGIWAXSLoader(md_naming_scheme=md_naming_scheme)

## Load & process raw data

### Load Xenocs data into list(s) of DataArrays, set your incident_angle!

In [ ]:
def load_xenocs_folders(sample_folders, files_to_load, incident_angle, phs_loader):
    """
    Loader for Xenocs Xeuss 3.0 GIWAXS output (Eiger2 1M detector)

    Inputs:
    - sample_folders (pathlib.path): A directory that contains all the xenocs data you want to load together (must all be same incident angle)
    - files_to_load (list): List of strings, chosen files to load for xenocs up to ['hi', 'lo', 'vd']
                            Do ['hi'] if you only took single images in the default detection
                            Do ['hi', 'lo'] or ['hi', 'lo', 'vd'] if you want to use stitching 
                            Note: 'vd' is the default stitched output for the xenocs, which is good but doesnt averate the two images together
    - incident_angle (float): Your incident angle, important for later transformations
    - phs_loader (phs.load.CMSGIWAXSLoader): PyHyperScattering CMSGIWAXSLoader object, with a metadata naming scheme ideally!

    Outputs:
    - loaded_DAs: dictionary with the xenocs 'files_to_load' type as keys for a list of corresponding DataArrays
    """

    # Load data in each Xenocs sample folder
    # Just uncomment corresponding lines if you don't want/have all the data files to load
    
    # Create dictionary with empty lists of each xenocs type to append to
    loaded_DAs = {}
    for file_to_load in files_to_load:
        loaded_DAs[file_to_load] = []
    
    # Load data into dictionary lists
    for sample_folder in sample_folders:
        for file_to_load in files_to_load:
            if file_to_load == 'hi':
                file = sorted(sample_folder.glob('*_00000.edf'))[0]
            elif file_to_load == 'lo':
                file = sorted(sample_folder.glob('*_00001.edf'))[0]
            elif file_to_load == 'vd':
                file = sorted(sample_folder.glob('*-00001.edf'))[0]
            else:
                print("invalide xenocs file_to_load option: ['hi', 'lo', 'vd']")
        
            DA = loader.loadSingleImage(file)
        
            DA.attrs['incident_angle'] = incident_angle
        
            DA = DA.where(DA!=-1)
        
            loaded_DAs[file_to_load].append(DA)

    return loaded_DAs

In [ ]:
incident_angle = 0.15  # Set your incident angle you used for this data!
files_to_load = ['hi', 'lo', 'vd']  # Choose types of xenocs images to load

loaded_DAs = load_xenocs_folders(sample_folders, files_to_load, incident_angle, loader)
loaded_DAs

hi_DAs = loaded_DAs['hi']
lo_DAs = loaded_DAs['lo']
vd_DAs = loaded_DAs['vd']

In [ ]:
# Optionally check the lists
display(hi_DAs)
display(len(hi_DAs))

### Combine two Xenocs Xeuss 3.0 detector images (stitch gap & average counts)
Benefit here versus the default 'vd' images from Xenocs: **average overlapping data!**

Xenocs Xeuss 3.0 'line eraser' moves det0xvd (0.75) and det0zvd (-4.125), corresponding to pixel shifts:

'lo_DA' (_00001.edf') compared to 'hi_DA' (_00000.edf): Detector moves 10 pixels to the right, 55 pixels down. 

As currently written, this code maintains the original hi_DA shape. The extra 55 pixels at the bottom and the extra 10 pixels to the right are thrown out.

#### Checking original data

In [ ]:
# fab_img = fabio.open(hi_file)
# fab_img.header
# (0.75e-3) / eiger2_1m  # use det0xvd shift: so a 10 pixel shift for the hi-lo gap?

In [ ]:
# plt.close('all')
# cmin = lo_DA.quantile(0.01)
# cmax = lo_DA.quantile(0.99)
# lo_DA.plot.imshow(cmap=plt.cm.turbo, norm=plt.Normalize(cmin,cmax), origin='upper')
# plt.show()

In [ ]:
# plt.close('all')
# cmin = hi_DA.quantile(0.01)
# cmax = hi_DA.quantile(0.99)
# hi_DA.plot.imshow(cmap=plt.cm.turbo, norm=plt.Normalize(cmin,cmax), origin='upper')
# plt.show()

In [ ]:
# # confirm that vd stitched file is NOT what we want 

# plt.close('all')
# fig, ax = plt.subplots()
# hi_DA.sel(pix_x=slice(10, 80), pix_y=slice(30, 605)).sum('pix_x').plot.line(ax=ax)
# vd_DA.sel(pix_x=slice(10, 80), pix_y=slice(30, 605)).sum('pix_x').plot.line(ax=ax, linestyle='--')
# plt.show()

#### Combine images & check

In [ ]:
# Load mask & hi_lo shift:
# Below are the correct values the xenocs xeuss 3.0 shift
y_shift = 55 
x_shift = 10

In [ ]:
# Stitch & average the two images
stitched_DAs = []
for tup in zip(lo_DAs, hi_DAs):
    lo_DA = tup[0].copy()
    hi_DA = tup[1].copy()
    
    # Shift lo DA to match hi da
    shifted_lo_DA = lo_DA.shift(pix_x=x_shift, pix_y=y_shift)

    # Stich missing data in hi_DA from shifted lo_DA:
    filled_hi_DA = hi_DA.combine_first(shifted_lo_DA)

    # Average data that is in both images:
    # Make boolean mask; true where there is overlap between images, false where there isn't
    overlap_mask = hi_DA.notnull() & shifted_lo_DA.notnull()  
    # Keep original combined data where there isn't overlap, average with shifted lo_DA otherwise
    stitched_DA = filled_hi_DA.where(~overlap_mask, (filled_hi_DA + shifted_lo_DA) / 2)   

    # Mask area below horizon, this will be above pix_y ~ 895:    
    stitched_DA = stitched_DA.where(stitched_DA.pix_y<895)
    
    # Record stitched DAs if good:
    stitched_DAs.append(stitched_DA)


    
# # If you want to double check where the overlap
# # It may likely be evident in the actual data too, where it appears smoother vs not
# overlap_mask.plot.imshow()
# plt.show()

In [ ]:
%matplotlib inline

In [ ]:
cmap.set_bad('black')

In [ ]:
# plt.close('all')
# Plot check stitched DAs
for stitched_DA in stitched_DAs[:]:
    cmin = stitched_DA.quantile(0.3)
    cmax = stitched_DA.quantile(0.99)
    ax = stitched_DA.plot.imshow(norm=plt.Normalize(cmin,cmax), origin='upper', cmap=cmap, interpolation='antialiased')
    ax.axes.set_title(f'{stitched_DA.film}')    
    plt.show()
    plt.close('all') 

### Apply rotation corrections if necessary
Not very refined yet. Requires defining a dictionary of bottom_line_points for each image to rotate. e.g. {'DataArray_identifier': [(x1,y1), (x2, y2)]}

In [ ]:
# from scipy import ndimage

In [ ]:
# # Interactively plot data of selected sample to identify point coordinates
# plt.close('all')
# DA = stitched_DAs[0].copy()

# cmin=DA.quantile(0.2)
# cmax=DA.quantile(0.98)
# ax = DA.plot.imshow(norm=plt.Normalize(cmin,cmax), origin='upper', cmap=cmap)
# ax.axes.set_title(f'{DA.film}')    
# plt.show()

In [ ]:
# # Create dictionary containing points which define line by which to calculate amount to rotate from
# bottom_line_points - {}

In [ ]:
# pivot_poni = (713, 1251)
# def rotateImage(img, angle, pivot):
#     padX = [img.shape[1] - pivot[0], pivot[0]]
#     padY = [img.shape[0] - pivot[1], pivot[1]]
#     imgP = np.pad(img, [padY, padX], 'constant')
#     imgR = ndimage.rotate(imgP, angle, reshape=False)
#     return imgR[padY[0] : -padY[1], padX[0] : -padX[1]]

In [ ]:
# # Apply rotation corrections based on lines drawn between points

# rot_corr_DAs = []
# for DA in tqdm(stitched_DAs, desc='Rotating...'):  
#     # Get line points from dictionary
#     p1, p2 = bottom_line_points[f'{DA.sample_id}_{DA.exposure_time}']

#     # Calculate the angle from points
#     dx = p2[0] - p1[0]
#     dy = p2[1] - p1[1]
#     angle_radians = np.arctan2(dy, dx)
#     angle_degrees = np.degrees(angle_radians)

#     # Rotate image & save into list
#     rot_corr_DA = xr.apply_ufunc(rotateImage, DA, angle_degrees, pivot_poni)
#     # rot_corr_DA = xr.apply_ufunc(ndimage.rotate, DA, angle_degrees, (1, 0), False, None, 3, 'constant')
#     # rot_corr_DA = xr.apply_ufunc(ndimage.rotate, DA, 0, (1, 0), False)
#     rot_corr_DA.attrs = DA.attrs
#     rot_corr_DAs.append(rot_corr_DA)
    
# # Plot check
# for rot_corr_DA in rot_corr_DAs:
#     cmin = rot_corr_DA.quantile(0.2)
#     cmax = rot_corr_DA.quantile(0.99)
#     rot_corr_DA.plot.imshow(norm=plt.Normalize(cmin,cmax), origin='upper', cmap=cmap)
#     plt.show()
#     plt.close('all')

### pygix-backed transformation

In [ ]:
# Initialize integrators
# Don't forget to ensure the correct incident angle is set
incident_angle = stitched_DAs[0].incident_angle
recip_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                                     ponifile = poniFile,
                                                     output_space = 'recip',
                                                     incident_angle = incident_angle)
caked_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                                     ponifile = poniFile,
                                                     output_space = 'caked',
                                                     incident_angle = incident_angle)

In [ ]:
# Use integrators to fill list of integrated DataArrays

recip_DAs = []
caked_DAs = []
for DA in tqdm(stitched_DAs, desc='Transforming to reciprocal space'):   
    # Integrate single image
    recip_DA = recip_integrator.integrateSingleImage(DA)
    caked_DA = caked_integrator.integrateSingleImage(DA)

    # Append to corresponding list
    recip_DAs.append(recip_DA)
    caked_DAs.append(caked_DA)

In [ ]:
%matplotlib inline
plt.close('all')

In [ ]:
plt.close('all')
DA = caked_DAs[0].copy()

cmin = DA.sel(qr=slice(0.25,None)).quantile(0.01)
cmax = DA.sel(qr=slice(0.25,None)).quantile(0.995)
DA.plot.imshow(norm=plt.Normalize(cmin,cmax), cmap=cmap)
plt.show()

In [ ]:
# Plot check recip DAs

# Set q bounds
qxy_min = -0.65
qxy_max = 2
qz_min = 0
qz_max = 2

for DA in recip_DAs:
# for DA in recip_DS.data_vars.values():
    sliced_DA = DA.sel(q_xy=slice(qxy_min, qxy_max), q_z=slice(qz_min,qz_max))
    
    cmin = sliced_DA.quantile(0.1)
    cmax = sliced_DA.quantile(0.993)
    ax = sliced_DA.plot.imshow(norm=plt.Normalize(cmin,cmax), cmap=cmap, figsize=(5,3.5), interpolation='antialiased')
    ax.axes.set_title(f'{sliced_DA.film}')   
    ax.axes.set(aspect='equal', xlabel='$Q_{xy}$ $[\\AA^{-1}]$', ylabel='$Q_{z}$ $[\\AA^{-1}]$')
    ax.axes.xaxis.set_minor_locator(plt.MultipleLocator(0.1))
    ax.axes.yaxis.set_minor_locator(plt.MultipleLocator(0.1))
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
    ax.figure.set(dpi=150)
    
    # # Save
    # savePath = outPath.joinpath('recip_images')
    # savePath.mkdir(exist_ok=True)
    # ax.figure.savefig(savePath.joinpath(f'{sliced_DA.material}_{sliced_DA.solvent}'), dpi=120)
    
    plt.show()
    plt.close('all') 

## Save / export processed data

### Save zarrs

In [ ]:
# Stack data into a Datasets first:
raw_DS = xr.Dataset({DA.attrs['film']: DA for DA in stitched_DAs})
recip_DS = xr.Dataset({DA.attrs['film']: DA for DA in recip_DAs})
caked_DS = xr.Dataset({DA.attrs['film']: DA for DA in caked_DAs})

In [ ]:
outPath

In [ ]:
# Check savePath
savePath = outPath.joinpath('AX_films_01/zarrs')
savePath.exists()

In [ ]:
# Save zarrs
recip_DS.to_zarr(savePath.joinpath('recip.zarr'), mode='w')
raw_DS.to_zarr(savePath.joinpath('raw.zarr'), mode='w')
caked_DS.to_zarr(savePath.joinpath('caked.zarr'), mode='w')

### Numpy binary format

In [ ]:
# sliced_recip_DA = recip_DA.sel(q_xy=slice(qxy_min, qxy_max), q_z=slice(qz_min,qz_max))

# display(sliced_recip_DA.data.shape)
# display(sliced_recip_DA.q_xy.data.shape)
# display(sliced_recip_DA.q_z.data.shape)

In [ ]:
# npysPath = outPath.joinpath('npys')
# savePath = npysPath.joinpath(f'{sample_name}')
# savePath.mkdir(exist_ok=True)

# np.save(savePath.joinpath(f'data_{sample_name}.npy'), sliced_recip_DA.data)
# np.save(savePath.joinpath(f'qxy_{sample_name}.npy'), sliced_recip_DA.q_xy.data)
# np.save(savePath.joinpath(f'qz_{sample_name}.npy'), sliced_recip_DA.q_z.data)

In [ ]:
# data = np.load(savePath.joinpath(f'data_{sample_name}.npy'))
# plt.imshow(data, norm=plt.Normalize(cmin,cmax), origin='lower')
# plt.show()

### CSVs

In [ ]:
# # A way to save data as csv files
# for DA in tqdm(DS.data_vars.values()):
#     # qxy columns, qz rows
#     DA.to_pandas().to_csv(outPath.joinpath('PM6-Y6_waxs', f'cartesian-2D_{DA.polymer}-{DA.weight_percent}_{DA.incident_angle}_{DA.scan_id}.csv'))

## Misc 

### Yoneda check:

In [ ]:
def qz(wavelength, alpha_crit, alpha_incidents):
    qz_inv_meters = ((4 * np.pi) / (wavelength)) * (np.sin(np.deg2rad((alpha_incidents + alpha_crit)/2)))
    # qz_inv_meters = ((4 * np.pi) / (wavelength)) * (np.sin(np.deg2rad(alpha_crit)) + np.sin(np.deg2rad(alpha_incidents)))
    qz_inv_angstroms = qz_inv_meters / 1e10
    return qz_inv_angstroms

wavelength = 1.5420919527363186e-10  # 8.04 keV
# wavelength = 9.762535309700809e-11  # 12.7 keV
# wavelength = 1.2398419843320025e-10  # 10 keV

alpha_crit = 0.11  # organic film critical angle
alpha_incidents = np.array([0.15])

yoneda_angles = alpha_incidents + alpha_crit

qz(wavelength, alpha_crit, alpha_incidents)

In [ ]:
# %matplotlib widget

In [ ]:
# ax = recip_DS['sam6_2s'].plot.imshow(norm=plt.Normalize(cmin,cmax), cmap=cmap)
# ax.axes.set_title(f'{sliced_DA.sample_id}: {sliced_DA.exposure_time}')   
# ax.axes.set(aspect='equal')
# plt.show()

In [ ]:
# def select_attrs(data_arrays_iterable, selected_attrs_dict):
#     """
#     Selects data arrays whose attributes match the specified values.

#     Parameters:
#     data_arrays_iterable: Iterable of xarray.DataArray objects.
#     selected_attrs_dict: Dictionary where keys are attribute names and 
#                          values are the attributes' desired values.

#     Returns:
#     List of xarray.DataArray objects that match the specified attributes.
#     """    
#     sublist = list(data_arrays_iterable)
    
#     for attr_name, attr_values in selected_attrs_dict.items():
#         sublist = [da for da in sublist if da.attrs[attr_name] in attr_values]
                
#     return sublist

In [ ]:
# # Yoneda peak linecut check
# qxy_min = 0.22
# qxy_max = 2
# qz_min = -0.02
# qz_max = 0.06

# selected_DAs = select_attrs(fixed_recip_DS.data_vars.values(), selected_attrs_dict)
# for DA in tqdm(selected_DAs):
#     # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
#     sliced_DA = DA.sel(q_xy=slice(qxy_min, qxy_max), q_z=slice(qz_min, qz_max))
#     qz_integrated_DA = sliced_DA.sum('q_xy')
    
#     # Plot
#     qz_integrated_DA.plot.line(label=DA.incident_angle)
    
# plt.legend()
# plt.grid(visible=True, which='major', axis='x')
# plt.show()